In [5]:
pip install vaex

  Using cached vaex-4.17.0-py3-none-any.whl.metadata (6.0 kB)
  Using cached vaex-core-4.17.1.tar.gz (2.5 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached vaex_astro-0.9.3-py3-none-any.whl.metadata (345 bytes)
  Using cached vaex_hdf5-0.14.1-py3-none-any.whl.metadata (321 bytes)
  Using cached vaex_viz-0.5.4-py3-none-any.whl.metadata (345 bytes)
  Using cached vaex_server-0.9.0-py3-none-any.whl.metadata (430 bytes)
  Using cached vaex_jupyter-0.8.2-py3-none-any.whl.metadata (517 bytes)
  Using cached vaex_ml-0.18.3-py3-none-any.whl.metadata (485 bytes)
  Using cached aplus-0.11.0-py3-none-any.whl
  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached future-1.0.0-py3-

  error: subprocess-exited-with-error
  
  × Building wheel for vaex-core (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [332 lines of output]
      <string>:4: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-311\vaex
      copying vaex\agg.py -> build\lib.win-amd64-cpython-311\vaex
      copying vaex\array_types.py -> build\lib.win-amd64-cpython-311\vaex
      copying vaex\asyncio.py -> build\lib.win-amd64-cpython-311\vaex
      copying vaex\benchmark.py -> build\lib.win-amd64-cpython-311\vaex
      copying vaex\cache.py -> build\lib.win-amd64-cpython-311\vaex
      copying vaex\column.py -> build\lib.win-amd64-cpython-311\vaex
      copying vaex\config.py -> build\lib.win-amd64-cpython-311\vaex
      copying vaex\convert.py -> build\lib.win-a

In [4]:
import os
import pandas as pd
import csv
from glob import glob
from pathlib import Path
from multiprocessing import Pool, cpu_count
from functools import partial
import vaex
base_dir = 'C:/Users/harsh/Downloads/aditya-l1-isro-main/ace_daily'
output_dir = 'C:/Users/harsh/Downloads/aditya-l1-isro-main/output_data'
os.makedirs(output_dir, exist_ok=True)

# File type configurations
file_types = {
    '_ace_epam_5m': {
        'suffix': '*_ace_epam_5m.txt',
        'skiprows': 14,
        'columns': [
            'YR', 'MO', 'DA', 'HHMM', 'Julian Day', 'Seconds of the Day',
            'Electron S', '38-53', '175-315', 'Proton S', '47-68', '115-195',
            '310-580', '795-1193', '1060-1900', 'Anis. Index'
        ],
        'na_values': ['-1.00e+05', '-1.00'],
        'dtypes': {
            'YR': 'int32', 'MO': 'int32', 'DA': 'int32', 'HHMM': 'str',
            'Julian Day': 'int32', 'Seconds of the Day': 'int32',
            'Electron S': 'int32', '38-53': 'float32', '175-315': 'float32',
            'Proton S': 'int32', '47-68': 'float32', '115-195': 'float32',
            '310-580': 'float32', '795-1193': 'float32', '1060-1900': 'float32',
            'Anis. Index': 'float32'
        }
    },
    '_ace_mag_1m': {
        'suffix': '*_ace_mag_1m.txt',
        'skiprows': 12,
        'columns': [
            'YR', 'MO', 'DA', 'HHMM', 'Julian Day', 'Seconds of the Day',
            'S', 'Bx', 'By', 'Bz', 'Bt', 'Lat.', 'Long.'
        ],
        'na_values': ['-999.9'],
        'dtypes': {
            'YR': 'int32', 'MO': 'int32', 'DA': 'int32', 'HHMM': 'str',
            'Julian Day': 'int32', 'Seconds of the Day': 'int32',
            'S': 'int32', 'Bx': 'float32', 'By': 'float32', 'Bz': 'float32',
            'Bt': 'float32', 'Lat.': 'float32', 'Long.': 'float32'
        }
    },
    '_ace_sis_5m': {
        'suffix': '*_ace_sis_5m.txt',
        'skiprows': 12,
        'columns': [
            'YR', 'MO', 'DA', 'HHMM', 'Julian Day', 'Seconds of the Day',
            'S (>10 MeV)', '>10 MeV', 'S (>30 MeV)', '>30 MeV'
        ],
        'na_values': ['-1.00e+05'],
        'dtypes': {
            'YR': 'int32', 'MO': 'int32', 'DA': 'int32', 'HHMM': 'str',
            'Julian Day': 'int32', 'Seconds of the Day': 'int32',
            'S (>10 MeV)': 'int32', '>10 MeV': 'float32',
            'S (>30 MeV)': 'int32', '>30 MeV': 'float32'
        }
    },
    '_ace_swepam_1m': {
        'suffix': '*_ace_swepam_1m.txt',
        'skiprows': 12,
        'columns': [
            'YR', 'MO', 'DA', 'HHMM', 'Julian Day', 'Seconds of the Day',
            'S', 'Proton Density', 'Bulk Speed', 'Ion Temperature'
        ],
        'na_values': ['-9999.9', '-1.00e+05'],
        'dtypes': {
            'YR': 'int32', 'MO': 'int32', 'DA': 'int32', 'HHMM': 'str',
            'Julian Day': 'int32', 'Seconds of the Day': 'int32',
            'S': 'int32', 'Proton Density': 'float32', 'Bulk Speed': 'float32',
            'Ion Temperature': 'float32'
        }
    }
}

def process_file_type(file_type, config):
    files = glob(os.path.join(base_dir, '*', config['suffix']))
    if not files:
        print(f"No files found for {file_type}")
        return
    dfs = []
    for file in files:
        df = vaex.from_csv(file, delim_whitespace=True, skip_lines=config['skiprows'], names=config['columns'])
        df['datetime'] = vaex.vdatetime(df['YR'].astype(str) + df['MO'].astype(str).str.zfill(2) +
                                       df['DA'].astype(str).str.zfill(2) + df['HHMM'].str.zfill(4),
                                       format='%Y%m%d%H%M')
        df['source_file'] = os.path.basename(file)
        dfs.append(df)
    if dfs:
        combined_df = vaex.concat(dfs)
        output_file = os.path.join(output_dir, f"{file_type[1:]}_data.parquet")
        combined_df.export_parquet(output_file)
        print(f"Saved {output_file} with {len(combined_df)} rows")



ModuleNotFoundError: No module named 'vaex'